In [1]:
# Import the dependencies.
import ipywidgets as widgets
widgets.IntSlider()
%matplotlib inline
import gmaps.datasets
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tiksi,71.6872,128.8694,17.62,96,100,7.36,RU,2021-10-19 16:39:00
1,1,Cherskiy,68.7500,161.3000,-13.85,100,98,3.60,RU,2021-10-19 16:39:00
2,2,Bredasdorp,-34.5322,20.0403,64.27,82,0,14.99,ZA,2021-10-19 16:39:01
3,3,Ushuaia,-54.8000,-68.3000,55.06,50,75,20.71,AR,2021-10-19 16:37:42
4,4,Rikitea,-23.1203,-134.9692,74.71,72,5,21.47,PF,2021-10-19 16:39:02


In [2]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [3]:
# Configure gmaps to use your Google API key.
map_client = gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Atuona,-9.8000,-139.0333,77.67,76,31,21.23,PF,2021-10-19 16:39:03
6,6,Mitsamiouli,-11.3847,43.2844,80.89,65,40,9.22,KM,2021-10-19 16:39:04
13,13,Faanui,-16.4833,-151.7500,79.48,78,99,13.62,PF,2021-10-19 16:35:21
17,17,Georgetown,5.4112,100.3354,78.75,94,40,5.01,MY,2021-10-19 16:38:07
27,27,Cuamba,-14.8031,36.5372,81.99,35,69,11.05,MZ,2021-10-19 16:39:12
28,28,Victoria,22.2855,114.1577,80.33,79,70,1.99,HK,2021-10-19 16:38:17
29,29,Tete,-16.1564,33.5867,89.69,37,40,11.50,MZ,2021-10-19 16:39:12
31,31,Camapua,-19.5314,-54.0439,84.09,56,88,6.29,BR,2021-10-19 16:39:13
39,39,Hithadhoo,-0.6000,73.0833,82.71,75,70,11.92,MV,2021-10-19 16:39:06
44,44,San Juan,-31.5375,-68.5364,83.21,13,0,7.18,AR,2021-10-19 16:39:18


In [9]:
preferred_cities_df.count()

City_ID       144
City          144
Lat           144
Lng           144
Max Temp      144
Humidity      144
Cloudiness    144
Wind Speed    144
Country       144
Date          144
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Atuona,PF,77.67,-9.8000,-139.0333,
6,Mitsamiouli,KM,80.89,-11.3847,43.2844,
13,Faanui,PF,79.48,-16.4833,-151.7500,
17,Georgetown,MY,78.75,5.4112,100.3354,
27,Cuamba,MZ,81.99,-14.8031,36.5372,
28,Victoria,HK,80.33,22.2855,114.1577,
29,Tete,MZ,89.69,-16.1564,33.5867,
31,Camapua,BR,84.09,-19.5314,-54.0439,
39,Hithadhoo,MV,82.71,-0.6000,73.0833,
44,San Juan,AR,83.21,-31.5375,-68.5364,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [18]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Atuona,PF,77.67,-9.8000,-139.0333,Villa Enata
6,Mitsamiouli,KM,80.89,-11.3847,43.2844,Foyer ADM
13,Faanui,PF,79.48,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
17,Georgetown,MY,78.75,5.4112,100.3354,Cititel Penang
27,Cuamba,MZ,81.99,-14.8031,36.5372,Hotel Vision


In [19]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
